In [163]:
import pandas as pd
from rapidfuzz.process import extractOne
import re
import os
import math


In [146]:

def load_projections(dir):
    positions = ["QB", "RB", "WR", "TE", "DEF"]
    dfs = {}
    for pos in positions:
        file = pos + "_projections.csv"
        if os.path.exists(os.path.join(dir, file)):
            dfs[pos] = pd.read_csv(os.path.join(dir, file))
    return dfs

In [172]:
df_sleeper = pd.read_csv('data/sleeper/all_players.csv')
print(df_sleeper.columns)
# print(df_sleeper["full_name"])

dict_sleeper = {
    "QB": df_sleeper.loc[df_sleeper['position'] == "QB"],
    "RB": df_sleeper.loc[df_sleeper['position'] == "RB"],
    "WR": df_sleeper.loc[df_sleeper['position'] == "WR"],
    "TE": df_sleeper.loc[df_sleeper['position'] == "TE"],
    "DEF": df_sleeper.loc[df_sleeper['position'] == "DEF"],
    "K": df_sleeper.loc[df_sleeper['position'] == "K"]
}




Index(['sleeper_id', 'first_name', 'team_abbr', 'rotowire_id', 'birth_state',
       'depth_chart_position', 'gsis_id', 'weight', 'practice_participation',
       'swish_id', 'active', 'high_school', 'practice_description', 'height',
       'fantasy_data_id', 'injury_start_date', 'birth_country', 'player_id',
       'years_exp', 'depth_chart_order', 'stats_id', 'number', 'status',
       'rotoworld_id', 'search_full_name', 'competitions', 'search_last_name',
       'age', 'injury_status', 'injury_notes', 'birth_date',
       'search_first_name', 'last_name', 'news_updated', 'team', 'yahoo_id',
       'fantasy_positions', 'college', 'oddsjam_id', 'hashtag', 'opta_id',
       'position', 'metadata', 'birth_city', 'pandascore_id', 'full_name',
       'espn_id', 'search_rank', 'sportradar_id', 'injury_body_part', 'sport',
       'Player Id', 'positional_rank', 'adp', 'adp_date', 'rank_int', 'id'],
      dtype='object')


In [173]:

# os.listdir("data/proj_scrape")

dict_dfs = {"QB": [], "RB": [], "WR": [], "TE": [], "DEF": [], "K": []}

dir = "data/proj_scrape/"

for root, dirs, files in os.walk(dir):
    for file in files:
        if re.match(r'[A-Z]{1,3}_projections.csv', file):
            df = pd.read_csv(os.path.join(root, file))
            data_src = re.sub(r'data/proj_scrape/', '', root)
            pos = re.sub(r'_projections.csv', '', file)
            dict_dfs[pos].append(df)
            

dict_dfs = {k: pd.concat(v).reset_index() for k, v in dict_dfs.items()}

for pos, df in dict_dfs.items():
    # print(pos)
    # if pos != "DEF":
    #     # df = df.rename(columns={"Team": "player"})
    #     continue
    df["name_match"] = df["player"].apply(lambda x: extractOne(x, dict_sleeper[pos]["full_name"], score_cutoff=80))
    df["sleeper_row"] = df["name_match"].apply(lambda x: x[2] if x else None)
    # add sleeper_id to each df in dict_dfs
    df["sleeper_id"] = df["sleeper_row"].apply(lambda x: dict_sleeper[pos].loc[x, "player_id"] if x and not math.isnan(x) else None)
    df.to_csv(f"data/{pos}_projections.csv", index=False)




ValueError: cannot insert level_0, already exists

337    HOU
338     NE
339    BAL
340    PIT
341    IND
342    ARI
343    SEA
344     LV
345    NYJ
346    DAL
347    WAS
348    LAC
349     TB
350    CLE
351    ATL
352    CAR
353    JAX
354    LAR
355     NO
356     GB
357    MIA
358    DET
359    BUF
360    PHI
361     SF
362    NYG
363    TEN
364    CHI
365    CIN
366    DEN
367     KC
368    MIN
Name: player_id, dtype: object

In [33]:
# df_sleeper.loc[:,["full_name", "position", "team", "sleeper_id"]]

df_cbs_qb = pd.concat([cbs["QB"], espn["QB"], fantasysharks["QB"], freedraftguide["QB"], nfl["QB"], numberfire["QB"]]).reset_index()
df_sleeper_qb = df_sleeper.loc[df_sleeper['position'] == 'QB'].copy().reset_index()

df_proj_qbs["name_match"] = df_proj_qbs["player"].apply(lambda x: extractOne(query = x,
                                                               choices = df_sleeper_qb['full_name'],
                                                               score_cutoff=90))

df_proj_qbs




In [35]:
display(df_sleeper_qb.loc[:, ['full_name', 'sleeper_id', 'name_match']])

df_proj_qbs['player'][61]

,full_name,sleeper_id,name_match
9,Joe Flacco,19.0,"(Joe Flacco, 100.0, 61)"
10,Drew Lock,5854.0,"(Drew Lock, 100.0, 62)"
13,Dak Prescott,3294.0,"(Dak Prescott, 100.0, 5)"
14,Matthew Stafford,421.0,"(Matthew Stafford, 100.0, 22)"
18,Joe Milton,11557.0,"(Joe Milton III, 95.0, 288)"
25,Sam Darnold,4943.0,"(Sam Darnold, 100.0, 34)"
40,Daniel Jones,5870.0,"(Daniel Jones, 100.0, 23)"
44,Geno Smith,1373.0,"(Geno Smith, 100.0, 21)"
45,Tua Tagovailoa,6768.0,"(Tua Tagovailoa, 100.0, 18)"
47,Jalen Hurts,6904.0,"(Jalen Hurts, 100.0, 0)"


'Joe Flacco'